# 第七章：建立聊天應用程式
## Azure OpenAI API 快速入門


## 概覽
本筆記本改編自 [Azure OpenAI Samples Repository](https://github.com/Azure/azure-openai-samples?WT.mc_id=academic-105485-koreyst)，當中亦包括可以存取 [OpenAI](notebook-openai.ipynb) 服務的筆記本。

Python OpenAI API 同樣可以配合 Azure OpenAI 使用，只需作出少量修改。你可以在這裡了解更多相關分別：[如何用 Python 切換 OpenAI 同 Azure OpenAI 端點](https://learn.microsoft.com/azure/ai-services/openai/how-to/switching-endpoints?WT.mc_id=academic-109527-jasmineg)

如需更多快速入門示例，請參考官方 [Azure OpenAI 快速入門文件](https://learn.microsoft.com/azure/cognitive-services/openai/quickstart?pivots=programming-language-studio&WT.mc_id=academic-105485-koreyst)


## 目錄

[概覽](../../../../07-building-chat-applications/python)  
[開始使用 Azure OpenAI 服務](../../../../07-building-chat-applications/python)  
[建立你的第一個提示](../../../../07-building-chat-applications/python)  

[使用案例](../../../../07-building-chat-applications/python)  
[1. 摘要文本](../../../../07-building-chat-applications/python)  
[2. 分類文本](../../../../07-building-chat-applications/python)  
[3. 產生新產品名稱](../../../../07-building-chat-applications/python)  
[4. 微調分類器](../../../../07-building-chat-applications/python)  
[5. 嵌入](../../../../07-building-chat-applications/python)

[參考資料](../../../../07-building-chat-applications/python)


### 開始使用 Azure OpenAI 服務

新用戶需要[申請存取權限](https://aka.ms/oai/access?WT.mc_id=academic-105485-koreyst)才能使用 Azure OpenAI 服務。  
獲批後，用戶可以登入 Azure 入口網站，建立 Azure OpenAI 服務資源，並透過 Studio 開始試用各種模型。

[快速入門的好資源](https://techcommunity.microsoft.com/blog/educatordeveloperblog/azure-openai-service-is-now-generally-available/3719177?WT.mc_id=academic-105485-koreyst)


### 建立你第一個提示  
呢個簡短練習會為你簡單介紹點樣向 OpenAI 模型提交提示，完成一個簡單任務「摘要」。

**步驟**:  
1. 喺你嘅 python 環境安裝 OpenAI library  
2. 載入標準輔助 library，並設定你為 OpenAI 服務建立嘅常用安全認證  
3. 揀一個適合你任務嘅模型  
4. 為模型建立一個簡單提示  
5. 將你嘅請求提交到模型 API！


> [!NOTE] 如果你喺 Codespaces 或者 Devcontainer 入面運行呢個 notebook，呢一步唔需要做。


In [ ]:
%pip install openai python-dotenv

In [ ]:
import os
from openai import AzureOpenAI
import numpy as np
from dotenv import load_dotenv
load_dotenv()

#validate data inside .env file

client = AzureOpenAI(
  api_key=os.environ['AZURE_OPENAI_API_KEY'],  # this is also the default, it can be omitted
  api_version = "2023-05-15"
  )

### 3. 搵啱嘅模型  
GPT-3.5-turbo 或 GPT-4 模型可以理解同生成自然語言。呢個服務提供四種模型能力，每種都有唔同嘅效能同速度，適合唔同嘅工作需要。

[Azure OpenAI 模型](https://learn.microsoft.com/azure/cognitive-services/openai/concepts/models?WT.mc_id=academic-105485-koreyst)


In [ ]:
# Select the General Purpose curie model for text
model = os.environ['AZURE_OPENAI_DEPLOYMENT']

## 4. 提示設計

「大型語言模型的神奇之處在於，透過在大量文本上訓練以減少預測錯誤，模型最終會學到對這些預測有用的概念。例如，佢哋會學識以下概念」(1):

* 點樣串字
* 文法點運作
* 點樣改寫句子
* 點樣答問題
* 點樣同人對話
* 點樣用唔同語言寫作
* 點樣寫程式
* 等等

#### 點樣控制大型語言模型
「喺所有輸入大型語言模型嘅方法之中，最有影響力嘅就係文字提示」(1)。

大型語言模型可以用幾種方式提示去產生輸出：

- 指令：直接話俾模型知你想要咩
- 補全：引導模型去補完你想要嘅開頭
- 示範：用例子俾模型睇你想要咩，可以係：
- 喺提示入面加幾個例子
- 喺微調訓練數據集中加幾百甚至幾千個例子

#### 創建提示有三個基本指引：

**清楚示範同說明**。無論用指令、例子，定係兩者結合，都要清楚表達你想要咩。如果你想模型將一個清單按字母順序排列，或者將一段文字按情感分類，就要俾佢睇到你想要咁做。

**提供高質素數據**。如果你想建立一個分類器，或者想模型跟住某個模式做，就要確保有足夠例子。記得校對你嘅例子——模型通常夠聰明，可以識穿基本嘅拼寫錯誤並俾你回應，但佢有時都會以為你係特登咁做，咁就會影響回應。

**檢查你嘅設定。** temperature 同 top_p 呢兩個設定會影響模型產生回應時有幾大隨機性。如果你問嘅問題只得一個正確答案，就應該將呢啲設定調低。如果你想要多啲唔同嘅答案，就可以調高啲。最多人搞錯嘅地方就係以為呢啲設定係「聰明度」或者「創意」控制。

來源：https://learn.microsoft.com/azure/ai-services/openai/overview


In [ ]:
# Create your first prompt
text_prompt = "Should oxford commas always be used?"

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":text_prompt},])

response.choices[0].message.content

In [ ]:

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":text_prompt},])

response.choices[0].message.content

## 總結文本  
#### 挑戰  
透過喺一段文字後面加上「tl;dr:」嚟總結內容。留意模型點樣可以無需額外指示就做到多種任務。你可以試下用比「tl;dr」更具描述性的提示，去改變模型嘅行為，同埋自訂你想要嘅總結方式(3)。

最近有研究顯示，先用大量文本語料庫做預訓練，再針對特定任務做微調，可以喺好多NLP任務同基準測試上有顯著提升。雖然呢種方法嘅架構通常唔針對特定任務，但都仲需要成千上萬個針對任務嘅微調數據集。相比之下，人類通常只需要少量例子或者簡單指示，就可以做到新嘅語言任務——而現時嘅NLP系統大多都仲未做到。喺呢度，我哋展示咗，擴大語言模型規模可以大大提升針對任務無關、少樣本學習嘅表現，有時甚至可以同之前最先進嘅微調方法媲美。

tl;dr


# 幾個常見用例的練習  
1. 摘要文本  
2. 分類文本  
3. 產生新產品名稱  
4. 嵌入向量  
5. 微調分類器


In [ ]:
prompt = "Recent work has demonstrated substantial gains on many NLP tasks and benchmarks by pre-training on a large corpus of text followed by fine-tuning on a specific task. While typically task-agnostic in architecture, this method still requires task-specific fine-tuning datasets of thousands or tens of thousands of examples. By contrast, humans can generally perform a new language task from only a few examples or from simple instructions - something that current NLP systems still largely struggle to do. Here we show that scaling up language models greatly improves task-agnostic, few-shot performance, sometimes even reaching competitiveness with prior state-of-the-art fine-tuning approaches.\n\ntl;dr"


In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},])

response.choices[0].message.content

## 分類文字  
#### 挑戰  
將項目分類到推理時提供的類別中。在以下例子中，我哋會喺提示(*playground_reference)中同時提供類別同要分類嘅文字。

客戶查詢：你好，我手提電腦鍵盤有一粒掣最近壞咗，我需要換一粒：

分類類別：


In [ ]:
prompt = "Classify the following inquiry into one of the following: categories: [Pricing, Hardware Support, Software Support]\n\ninquiry: Hello, one of the keys on my laptop keyboard broke recently and I'll need a replacement:\n\nClassified category:"
print(prompt)

In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},])

response.choices[0].message.content

## 產生新產品名稱
#### 挑戰
根據範例字詞創作產品名稱。我哋會喺提示入面加入關於要為邊款產品起名嘅資訊，亦會提供一個類似嘅例子，展示我哋期望收到嘅命名模式。我哋仲將 temperature 數值調高，令結果更有創意同多變。

產品描述：家用奶昔機
種子字詞：快、健康、細小。
產品名稱：HomeShaker、Fit Shaker、QuickShake、Shake Maker

產品描述：一對可以適合任何腳型嘅鞋。
種子字詞：適應、合腳、全適配。


In [ ]:
prompt = "Product description: A home milkshake maker\nSeed words: fast, healthy, compact.\nProduct names: HomeShaker, Fit Shaker, QuickShake, Shake Maker\n\nProduct description: A pair of shoes that can fit any foot size.\nSeed words: adaptable, fit, omni-fit."

print(prompt)

In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt}])

response.choices[0].message.content

## 嵌入向量
本節會介紹如何取得嵌入向量，以及如何比較詞語、句子和文件之間的相似度。要運行以下的筆記本，你需要部署一個以 `text-embedding-ada-002` 為基礎模型的服務，並在 .env 檔案中設定它的部署名稱，使用 `AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT` 這個變數。


### 模型分類法 - 選擇嵌入模型

**模型分類法**：{family} - {capability} - {input-type} - {identifier}  

{family}     --> text-embedding  （嵌入模型系列）  
{capability} --> ada             （所有其他嵌入模型將於2024年退役）  
{input-type} --> n/a             （只適用於搜尋模型時指定）  
{identifier} --> 002             （版本002）  

model = 'text-embedding-ada-002'


> [!NOTE] 如果你在 Codespaces 或 Devcontainer 上運行此筆記本，以下步驟則無需執行


In [ ]:
# Dependencies for embeddings_utils
%pip install matplotlib plotly scikit-learn pandas

In [ ]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

In [ ]:
text = 'the quick brown fox jumped over the lazy dog'
model= os.environ['AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT']
client.embeddings.create(input='[text]', model=model).data[0].embedding

In [ ]:

# compare several words
automobile_embedding  = client.embeddings.create(input='automobile', model=model).data[0].embedding
vehicle_embedding     = client.embeddings.create(input='vehicle', model=model).data[0].embedding
dinosaur_embedding    = client.embeddings.create(input='dinosaur', model=model).data[0].embedding
stick_embedding       = client.embeddings.create(input='stick', model=model).data[0].embedding

print(cosine_similarity(automobile_embedding, vehicle_embedding))
print(cosine_similarity(automobile_embedding, dinosaur_embedding))
print(cosine_similarity(automobile_embedding, stick_embedding))

## 比較 cnn daily news 數據集嘅文章
來源: https://huggingface.co/datasets/cnn_dailymail


In [ ]:
import pandas as pd
cnn_daily_articles = ['BREMEN, Germany -- Carlos Alberto, who scored in FC Porto\'s Champions League final victory against Monaco in 2004, has joined Bundesliga club Werder Bremen for a club record fee of 7.8 million euros ($10.7 million). Carlos Alberto enjoyed success at FC Porto under Jose Mourinho. "I\'m here to win titles with Werder," the 22-year-old said after his first training session with his new club. "I like Bremen and would only have wanted to come here." Carlos Alberto started his career with Fluminense, and helped them to lift the Campeonato Carioca in 2002. In January 2004 he moved on to FC Porto, who were coached by José Mourinho, and the club won the Portuguese title as well as the Champions League. Early in 2005, he moved to Corinthians, where he impressed as they won the Brasileirão,but in 2006 Corinthians had a poor season and Carlos Alberto found himself at odds with manager, Emerson Leão. Their poor relationship came to a climax at a Copa Sul-Americana game against Club Atlético Lanús, and Carlos Alberto declared that he would not play for Corinthians again while Leão remained as manager. Since January this year he has been on loan with his first club Fluminense. Bundesliga champions VfB Stuttgart said on Sunday that they would sign a loan agreement with Real Zaragoza on Monday for Ewerthon, the third top Brazilian player to join the German league in three days. A VfB spokesman said Ewerthon, who played in the Bundesliga for Borussia Dortmund from 2001 to 2005, was expected to join the club for their pre-season training in Austria on Monday. On Friday, Ailton returned to Germany where he was the league\'s top scorer in 2004, signing a one-year deal with Duisburg on a transfer from Red Star Belgrade. E-mail to a friend .',
                        '(CNN) -- Football superstar, celebrity, fashion icon, multimillion-dollar heartthrob. Now, David Beckham is headed for the Hollywood Hills as he takes his game to U.S. Major League Soccer. CNN looks at how Bekham fulfilled his dream of playing for Manchester United, and his time playing for England. The world\'s famous footballer has begun a five-year contract with the Los Angeles Galaxy team, and on Friday Beckham will meet the press and reveal his new shirt number. This week, we take an in depth look at the life and times of Beckham, as CNN\'s very own "Becks," Becky Anderson, sets out to examine what makes the man tick -- as footballer, fashion icon and global phenomenon. It\'s a long way from the streets of east London to the Hollywood Hills and Becky charts Beckham\'s incredible rise to football stardom, a journey that has seen his skills grace the greatest stages in world soccer. She goes in pursuit of the current hottest property on the sports/celebrity circuit in the U.S. and along the way explores exactly what\'s behind the man with the golden boot. CNN will look back at the life of Beckham, the wonderfully talented youngster who fulfilled his dream of playing for Manchester United, his marriage to pop star Victoria, and the trials and tribulations of playing for England. We\'ll look at the highs (scoring against Greece), the lows (being sent off during the World Cup), the Man. U departure for the Galacticos of Madrid -- and now the Home Depot stadium in L.A. We\'ll ask how Beckham and his family will adapt to life in Los Angeles -- the people, the places to see and be seen and the celebrity endorsement. Beckham is no stranger to exposure. He has teamed with Reggie Bush in an Adidas commercial, is the face of Motorola, is the face on a PlayStation game and doesn\'t need fashion tips as he has his own international clothing line. But what does the star couple need to do to become an accepted part of Tinseltown\'s glitterati? The road to major league football in the U.S.A. is a well-worn route for some of the world\'s greatest players. We talk to some of the former greats who came before him and examine what impact these overseas stars had on U.S. soccer and look at what is different now. We also get a rare glimpse inside the David Beckham academy in L.A, find out what drives the kids and who are their heroes. The perception that in the U.S.A. soccer is a "game for girls" after the teenage years is changing. More and more young kids are choosing the European game over the traditional U.S. sports. E-mail to a friend .',
                        'LOS ANGELES, California (CNN) -- Youssif, the 5-year-old burned Iraqi boy, rounded the corner at Universal Studios when suddenly the little boy hero met his favorite superhero. Youssif has always been a huge Spider-Man fan. Meeting him was "my favorite thing," he said. Spider-Man was right smack dab in front of him, riding a four-wheeler amid a convoy of other superheroes. The legendary climber of buildings and fighter of evil dismounted, walked over to Youssif and introduced himself. Spidey then gave the boy from a far-away land a gentle hug, embracing him in his iconic blue and red tights. He showed Youssif a few tricks, like how to shoot a web from his wrist. Only this time, no web was spun. "All right Youssif!" Spider-Man said after the boy mimicked his wrist movement. Other superheroes crowded around to get a closer look. Even the Green Goblin stopped his villainous ways to tell the boy hi. Youssif remained unfazed. He didn\'t take a liking to Spider-Man\'s nemesis. Spidey was just too cool. "It was my favorite thing," the boy said later. "I want to see him again." He then felt compelled to add: "I know it\'s not the real Spider-Man." This was the day of dreams when the boy\'s nightmares were, at least temporarily, forgotten. He met SpongeBob, Lassie and a 3-year-old orangutan named Archie. The hairy, brownish-red primate took to the boy, grabbing his hand and holding it. Even when Youssif pulled away, Archie would inch his hand back toward the boy\'s and then snatch it. See Youssif enjoy being a boy again » . The boy giggled inside a play area where sponge-like balls shot out of toy guns. It was a far different artillery than what he was used to seeing in central Baghdad, as recently as a week ago. He squealed with delight and raced around the room collecting as many balls as he could. He rode a tram through the back stages at Universal Studios. At one point, the car shook. Fire and smoke filled the air, debris cascaded down and a big rig skidded toward the vehicle. The boy and his family survived the pretend earthquake unscathed. "Even I was scared," the dad said. "Well, I wasn\'t," Youssif replied. The father and mother grinned from ear to ear throughout the day. Youssif pushed his 14-month-old sister, Ayaa, in a stroller. "Did you even need to ask us if we were interested in coming here?" Youssif\'s father said in amazement. "Other than my wedding day, this is the happiest day of my life," he said. Just a day earlier, the mother and father talked about their journey out of Iraq and to the United States. They also discussed that day nine months ago when masked men grabbed their son outside the family home, doused him in gas and set him on fire. His mother heard her boy screaming from inside. The father sought help for his boy across Baghdad, but no one listened. He remembers his son\'s two months of hospitalization. The doctors didn\'t use anesthetics. He could hear his boy\'s piercing screams from the other side of the hospital. Watch Youssif meet his doctor and play with his little sister » . The father knew that speaking to CNN would put his family\'s lives in jeopardy. The possibility of being killed was better than seeing his son suffer, he said. "Anything for Youssif," he said. "We had to do it." They described a life of utter chaos in Baghdad. Neighbors had recently given birth to a baby girl. Shortly afterward, the father was kidnapped and killed. Then, there was the time when some girls wore tanktops and jeans. They were snatched off the street by gunmen. The stories can be even more gruesome. The couple said they had heard reports that a young girl was kidnapped and beheaded --and her killers sewed a dog\'s head on the corpse and delivered it to her family\'s doorstep. "These are just some of the stories," said Youssif\'s mother, Zainab. Under Saddam Hussein, there was more security and stability, they said. There was running water and electricity most of the time. But still life was tough under the dictator, like the time when Zainab\'s uncle disappeared and was never heard from again after he read a "religious book," she said. Sitting in the parking lot of a Target in suburban Los Angeles, Youssif\'s father watched as husbands and wives, boyfriends and girlfriends, parents and their children, came and went. Some held hands. Others smiled and laughed. "Iraq finished," he said in what few English words he knows. He elaborated in Arabic: His homeland won\'t be enjoying such freedoms anytime soon. It\'s just not possible. Too much violence. Too many killings. His two children have only seen war. But this week, the family has seen a much different side of America -- an outpouring of generosity and a peaceful nation at home. "It\'s been a dream," the father said. He used to do a lot of volunteer work back in Baghdad. "Maybe that\'s why I\'m being helped now," the father said. At Universal Studios, he looked out across the valley below. The sun glistened off treetops and buildings. It was a picturesque sight fit for a Hollywood movie. "Good America, good America," he said in English. E-mail to a friend . CNN\'s Arwa Damon contributed to this report.'
]

cnn_daily_article_highlights = ['Werder Bremen pay a club record $10.7 million for Carlos Alberto .\nThe Brazilian midfielder won the Champions League with FC Porto in 2004 .\nSince January he has been on loan with his first club, Fluminense .',
                                'Beckham has agreed to a five-year contract with Los Angeles Galaxy .\nNew contract took effect July 1, 2007 .\nFormer English captain to meet press, unveil new shirt number Friday .\nCNN to look at Beckham as footballer, fashion icon and global phenomenon .',
                                'Boy on meeting Spider-Man: "It was my favorite thing"\nYoussif also met SpongeBob, Lassie and an orangutan at Universal Studios .\nDad: "Other than my wedding day, this is the happiest day of my life"'
]

cnn_df = pd.DataFrame({"articles":cnn_daily_articles, "highligths":cnn_daily_article_highlights})

cnn_df.head()

In [ ]:
article1_embedding    = client.embeddings.create(input=cnn_df.articles.iloc[0], model=model).data[0].embedding
article2_embedding    = client.embeddings.create(input=cnn_df.articles.iloc[1], model=model).data[0].embedding
article3_embedding    = client.embeddings.create(input=cnn_df.articles.iloc[2], model=model).data[0].embedding

print(cosine_similarity(article1_embedding, article2_embedding))
print(cosine_similarity(article1_embedding, article3_embedding))

# 參考資料  
- [Azure 文件 - Azure OpenAI 模型](https://learn.microsoft.com/azure/cognitive-services/openai/concepts/models?WT.mc_id=academic-105485-koreyst)  
- [OpenAI Studio 範例](https://oai.azure.com/portal?WT.mc_id=academic-105485-koreyst)


# 需要更多協助

[OpenAI 商業化團隊](AzureOpenAITeam@microsoft.com)


# 貢獻者
* Louis Li



---

**免責聲明**：  
本文件經由 AI 翻譯服務 [Co-op Translator](https://github.com/Azure/co-op-translator) 翻譯。我們致力於確保翻譯準確，但請注意，自動翻譯可能會出現錯誤或不準確之處。原始語言的文件應被視為具權威性的來源。如涉及重要資訊，建議尋求專業人工翻譯。本翻譯所引致的任何誤解或曲解，我們概不負責。
